In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html

import jupyterlab_dash
import dash
import dash_table
import dash_html_components as html

from pathlib import Path

import plotly.express as px

import plotly.graph_objs as go

from dash.dependencies import Input, Output

import pandas as pd

In [ ]:
base_dir = Path().cwd().parent
processed_dir = base_dir/'data'/'processed'/'daily_report'

days = pd.date_range('01/22/2020', '04/03/2020', normalize=True)
days = days.strftime('%m-%d-%Y')

df = pd.DataFrame()
for day in days:
    temp = pd.read_csv(processed_dir/f'{day}.csv')
    temp.loc[:, 'date'] = pd.to_datetime(day)
    df = df.append(temp)

## generate dataframe

In [ ]:
def generate_dataframe():
    
    df = pd.DataFrame()  

    for day in days:
        temp = pd.read_csv(processed_dir/f'{day}.csv')
        temp.loc[:, 'date'] = pd.to_datetime(day)
        df = df.append(temp)
        
    return df

In [ ]:
df = generate_dataframe()

## generate date picker

In [ ]:
def generate_date_picker():   
    return dcc.DatePickerSingle(
        id='date-picker',
        min_date_allowed=dt(2020,3,1),
        max_date_allowed=dt(2020,4,3),
        initial_visible_month=dt(2020,3,15),
        date=str(dt(2020, 3, 15))
    )

## generate map

In [ ]:
def generate_map(df, date=None):
    
    if date is not None:
        df_map = df[df['date'] == pd.to_datetime(date)]     
    else:
        df_map = df[df['date'] == pd.to_datetime('03-15-2020')]
    
    choro = go.Choropleth(
        locations=df_map['country'],
        locationmode='country names',
        z=df_map['confirmed'],
        text=df_map['country'],
        autocolorscale=False,
        colorscale="YlOrRd",
        showscale=False,
    )
    
    fig = go.Figure(choro)
    
    fig.update_layout(
        # title_text = 'World Map',
        geo_scope='world',
        margin={"r":10,"t":10,"l":10,"b":10},        
    )
    
    fig.update_geos(
        resolution=110,
        showcoastlines=True, coastlinecolor="black",
        showland=False, landcolor="LightGreen",
        showocean=True, oceancolor="LightBlue",
        showlakes=False, lakecolor="Blue",
        showrivers=False, rivercolor="Blue",
        showframe=False
    )
        
    return fig

## generate confirmed case development

In [ ]:
def generate_ccd(df):
    
    def get_min_mask(data, factor):
        mask = data.ge(factor)
        return mask

    df_pivot = df.pivot(index='date', columns='country', values=['confirmed'])
    
    country_list = [
        'Germany',
        'US',
        'France',
        'Spain',
        'Italy',
        'Belgium',
        'Denmark',
        'Switzerland'
    ]
    
    fig = go.Figure()

    for country in country_list:
    
        # select country
        mask = df_pivot.columns.get_level_values(1) == country
        df_c = df_pivot.iloc[:, mask]

        # transform dataframe to series
        df_c = df_c.squeeze()

        # get disease outbreak day
        mask_min = get_min_mask(df_c, 100)
        df_c = df_c[mask_min]
        df_c = df_c.reset_index(drop=True)

        # create graph
        graph = go.Scatter(
            x=df_c.index,
            y=df_c,
            name=country,
            mode='lines+markers'
        )
    
        # add graph to figure
        fig.add_trace(graph)
    
    fig.update_layout(
        # title_text = 'World Map',
        margin={"r":10,"t":10,"l":10,"b":10},
    )

    return fig

## generate table

In [ ]:
def generate_table(df,date=None):
    
    if date is not None:
        df = df[df['date'] == pd.to_datetime(date)]     
    else:
        df = df[df['date'] == pd.to_datetime('04-04-2020')]
    
    columns = ['country', 'confirmed', 'deaths', 'recovered']
    
    df = df[columns].copy()
    df = df.sort_values(by=columns[1], ascending=False)
    
    table = dash_table.DataTable(
        id='table-info',
        columns=[{"name": i.capitalize(), "id": i} for i in df[columns].columns],
        data=df.to_dict('records'),
        style_table={'height': '300px',
                     'overflowY': 'scroll',
                     'border': 'thin lightgrey solid'},
        page_action='none'
)
    
    return table

## generate country picker

In [ ]:
def generate_country_picker():
    
    checklist = dcc.Dropdown(
    options=[
        {'label': 'New York City', 'value': 'NYC'},
        {'label': 'Montreal', 'value': 'MTL'},
        {'label': 'San Francisco', 'value': 'SF'}
    ],
    value=['MTL', 'NYC'],
    multi=True
)      
    return checklist

In [ ]:
from datetime import datetime as dt
import dash
from dash.dependencies import Input, Output
import dash_html_components as html
import dash_core_components as dcc
import re

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    # set page header
    html.H4(children='COVID-Dashboard'),
    # 1st row place date picker
    html.Div(
        children=generate_date_picker(),
        style={
            'width': '50%', 
            'display': 'flex',
            'marginLeft': 24,
            'marginRight': 0,
            'marginTop': 0,
            'marginBottom': 10,
            'backgroundColor': '#a10b0b',
        }
    ),
    # 2nd row place map and country table
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            # place map graph 
            html.Div(
                dcc.Graph(id='graph-map', figure=generate_map(df)),
                style={
                    'width': '66%', 
                    'display': 'inline-block',
                    'marginLeft': 0,
                    'marginRight': 10,
                    'marginTop': 0,
                    'marginBottom': 10,
                    'backgroundColor': '#a10b0b',}
            ),
            # place country table
            html.Div(
                generate_table(df),
                style={
                    'width': '32%', 
                    'display': 
                    'inline-block', 
                    'marginLeft': 0,
                    'marginRight': 10,
                    'marginTop': 10,
                    'marginBottom': 10,
                    # 'backgroundColor': '#a10b0b',
                }
            )
        ],
    ),

    # 3rd row place timeseries and country selector
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            # place timeseries
            html.Div(
                dcc.Graph(
                    id='timeseries',
                    figure=generate_ccd(df)),
                style={
                    'width': '66%', 
                    'display': 'inline-block',
                    'marginLeft': 10,
                    'marginRight': 10,
                    'marginTop': 10,
                    'marginBottom': 10,
                    'backgroundColor': '#a10b0b',
                }
            ),

            # place country selector
            html.Div(
                generate_country_picker(),
                style={
                    'width': '32%',
                    'display': 'inline-block',
                    'marginLeft': 10,
                    'marginRight': 10,
                    'marginTop': 10,
                    'marginBottom': 10,
                    'backgroundColor': '#a10b0b',
                    'border': 'blue',
                }
            ),
        ],
    )
])


@app.callback(
    Output('graph-map', 'figure'),
     # Output('table-info', 'figure')
    [Input('date-picker', 'date')])
def update_output(date):
    map_graph = generate_map(df, date=pd.to_datetime(date))
    # table_info = generate_table(df, date=pd.to_datetime(date))

    return map_graph  # , table_info


viewer = jupyterlab_dash.AppViewer()
viewer.show(app)